In [2]:
# combine data of each day and make a single dataFrame  

In [25]:
using CSV
using DataFrames
using Dates
using Statistics
using Plots

In [50]:
# Local machine path to raw data
Node53_path = "/Users/prabu/Desktop/Joppa/Raw_data/LoRa_53"
Node54_path = "/Users/prabu/Desktop/Joppa/Raw_data/LoRa_54"

node53_ID = "47cb55800041001c"
node54_ID = "472b544e0028002b";

In [51]:
nodeId = node53_ID
path = Node53_path

years_path = filter(isdir,readdir(path,join=true)) 
years = basename.(years_path);

In [52]:
sensorName = ["BME280","IPS7100", "GPGGALR"];

In [53]:
function ss(dirr, csvv, sensr)
    df = CSV.read(dirr, DataFrame)
    
    try
    	df.dateTime =  SubString.(string.(df.dateTime), 1, 19) 
    	df.dateTime = DateTime.(df.dateTime,"yyyy-mm-dd HH:MM:SS")  
        #println(df)
    	df.dateTime = map((x) -> round(x, Dates.Minute(10)), df.dateTime)  
    	gdf = groupby(df, :dateTime)  
    	cgdf = combine(gdf, valuecols(gdf) .=> mean)
    
    	colFullName = names(cgdf)
    	for x in 2:length(colFullName)                                      # Starting from 2nd column and go through all the columns           
            num_let = length(colFullName[x])                                # taking number of characters in one column name
            colName = SubString.(string.(colFullName[x]), 1, num_let-5)     # removing last five characters of each column name (that is _mean)
            rename!(cgdf,colFullName[x] => sensr*"_"*colName)               # adding sensor name to the column name
    	end
    	return cgdf  
    	
    catch e println("Error on "*csvv) end

end;

In [61]:
n = 0
for sensor in sensorName
    k = 0
    for y in years
        months = readdir(path*"/"*y)
        for m in months
            try days = readdir(path*"/"*y*"/"*m)
                for d in days
                    try filess = readdir(path*"/"*y*"/"*m*"/"*d)
                        CSV_file = "MINTS_"*nodeId*"_"*sensor*"_"*y*"_"*m*"_"*d*".csv"
                        
                        if (CSV_file in filess)
                            dir = path*"/"*y*"/"*m*"/"*d*"/"*CSV_file
                            averaged_df = ss(dir, CSV_file, sensor)
                            df2 = averaged_df
                            k = k + 1
                            if (k > 1) global new_df = outerjoin(new_df, df2, on = intersect(names(new_df), names(df2)))
                            else new_df = averaged_df end
                        else println("No data of "*sensor) end
                        
                    catch e println("error on "*d) end
                end
            catch e println("error on "*m) end
        end
    end

    global n = n + 1
    if (n > 1) global final_df = outerjoin(final_df, new_df, on = :dateTime, makeunique=true) 
    else final_df = new_df end
end

error on .DS_Store
error on .DS_Store
error on .DS_Store
error on .DS_Store
error on .DS_Store
error on .DS_Store
error on .DS_Store
error on .DS_Store
error on .DS_Store
error on .DS_Store
error on .DS_Store
error on .DS_Store


In [62]:
final_df2 = sort!(final_df)

select!(final_df2, Not([:IPS7100_pc0_1, :IPS7100_pc0_3, :IPS7100_pc0_5, :IPS7100_pc1_0, :IPS7100_pc2_5, :IPS7100_pc5_0, 
            :IPS7100_pc10_0, :GPGGALR_Speed, :GPGGALR_Course, :GPGGALR_HDOP, :GPGGALR_Year, :GPGGALR_Month, :GPGGALR_Day,
            :GPGGALR_Hour, :GPGGALR_Minute, :GPGGALR_Second]))
#delete!(final_df2,[:IPS7100_pc0_1,:IPS7100_pc0_3])

for col in names(final_df2)
    if nrow(final_df2) == sum(ismissing.(final_df2[!, col])) select!(final_df2, Not(col)) end
end

CSV.write("/Users/prabu/Desktop/Joppa/df_lora53.csv", final_df2)

"/Users/prabu/Desktop/Joppa/df_lora53.csv"